In [ ]:
from lsst.daf.butler import Butler
from lsst.source.injection import CoaddInjectConfig,CoaddInjectTask
# NOTE: For injections into other dataset types, use the following instead:
# from lsst.source.injection import ExposureInjectConfig,ExposureInjectTask
# from lsst.source.injection import VisitInjectConfig,VisitInjectTask

# Instantiate a butler.
butler = Butler(REPO)

# Load an input deepCoadd dataset.
dataId = dict(
    instrument="HSC",
    skymap="hsc_rings_v1",
    tract=9813,
    patch=42,
    band="i",
)


visit_summary = butler.get(
     "finalVisitSummary",
     dataId=dataId,
     collections=INPUT_DATA_COLL)


# NOTE: Visit-level injections should instead use the visit summary table.
detector_summary = visit_summary.find(dataId["detector"])
psf = detector_summary.getPsf()
photo_calib = detector_summary.getPhotoCalib()
wcs = detector_summary.getWcs()

# Load input injection catalogs, here just for i-band catalogs.
injection_refs = butler.registry.queryDatasets(
    "injection_catalog",
    band="i",
    collections=INJECTION_CATALOG_COLL,
)
injection_catalogs = [
    butler.get(injection_ref) for injection_ref in injection_refs
]

# Instantiate the injection classes.
inject_config = CoaddInjectConfig()
inject_task = CoaddInjectTask(config=inject_config)

# Run the source injection task.
injected_output = inject_task.run(
    injection_catalogs=injection_catalogs,
    input_exposure=input_exposure.clone(),
    psf=psf,
    photo_calib=photo_calib,
    wcs=wcs,
)
injected_exposure=injected_output.output_exposure
injected_catalog=injected_output.output_catalog